In [1]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def get_ner(sentence):
    result = dependency_parser(sentence, properties={
        "outputFormat": "json",
        "annotators": "ner"})
    res = []
    for obj in result['sentences'][0]['entitymentions']:
        res.append((obj['text'], obj['ner']))
    return res

In [2]:
get_ner("you owe obama $4 in indonesia")

[('$4', 'MONEY'), ('indonesia', 'COUNTRY')]

In [3]:
import pandas as pd
from tqdm import  tqdm_notebook

def run():
    df = pd.read_csv("dataset/res16_baru.csv")
    sf = pd.DataFrame(columns = ['id','review','target','category','term'])
    count = 0
    all_ner = []
    money_ner = []
    title_ner = []
    for index in tqdm_notebook(range(len(df))):
        new_term = []
        ner = get_ner(df['review'][index])
        if len(ner):
#             print(df['review'][index])
#             print(ner)
#             print()
            for word, tipe in ner:
                if tipe != 'MONEY':
                    all_ner.append(tipe.lower())
                    all_ner.append(word)
                elif tipe == 'MONEY':
                    money_ner.append(tipe.lower())
                    money_ner.append(word)
    return all_ner, money_ner, title_ner
#         for nn in get_nouns(df['review'][index]):
#             new_term.append(nn)
#         if len(new_term) == 0:
#             new_term = '|'.join(get_nouns(df['review'][index]))
#             count += 1
        
#         sf = sf.append({'id': df['id'][index], 
#                     'review': df['review'][index],
#                     'category': df['category'][index],
#                     'term': '|'.join(new_term)
#                    }, ignore_index=True)
#     print(count)

#     sf.to_csv("Results/add_noun/"+ tipe +".csv")
#     sf.to_excel("Results/add_noun/"+ tipe +".xlsx")

In [4]:
new_ner, mon_ner, tit_ner = run()

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

In [5]:
splitted_ner = []
for ner in new_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())
ner_ner = sorted(list(dict.fromkeys(splitted_ner)))

In [6]:
print(ner_ner)

['100', '13', '15', '17', '2', '2nd', '2\xa01/2', '300', '4', '5', '50', '6th', '8pm.', '9:30', 'a', 'ago', 'america', 'avenue', 'baluchi', 'bart', 'bbe', 'bombay', 'brooklyn', 'casa', 'chef', 'chinatown', 'chinese', 'chutney', 'city', 'citysearch', 'co.', 'cobb', 'college', 'companion', 'country', 'current', 'date', 'day', 'dj', 'duration', 'egyptian', 'emilio', 'european', 'evening', 'every', 'farms', 'femme', 'few', 'first', 'flatbush', 'fornino', 'four', 'french', 'frickin', 'friday', 'gigondas', 'hampton', 'hand', 'he', 'her', 'him', 'hostess', 'hours', 'hudson', 'hutner', 'indian', 'italian', 'japanese', 'la', 'last', 'late', 'lobster', 'location', 'manager', 'manhattan', 'misc', 'months', "mother's", 'murray', 'nationality', 'new', 'night', 'nights', 'noon', 'now', 'number', 'ny', 'once', 'one', 'onion', 'ordinal', 'organization', 'owner', 'pam', 'paris', 'parisian', 'patroon', 'person', 'philly', 'present', 'raga', 'recently', 'reviewer', 'rice', 'river', 'rubber', 'salad', 'sa

In [7]:
splitted_ner = []
for ner in mon_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())
new_mon = sorted(list(dict.fromkeys(splitted_ner)))

In [9]:
import pandas as pd

def run_filter(tipe, where, ner):
    df = pd.read_csv("Results/" + where + tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'category','term', 'polarity'])
    for index, concat_terms in enumerate(df['term']):
        new_terms = []
        try:
            terms = concat_terms.split('|')
        except:
            terms = []
        for term in terms:
#             if term == 'time':
#                 print("   ")
            if term not in ner and len(term) > 3:
                new_terms.append(term)
        sentence = df['review'][index].lower()
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'target': df['target'][index],
                        'category': df['category'][index],
                        'term': '|'.join(new_terms),
                        'polarity': df['polarity'][index]
                       }, ignore_index=True)

    sf.to_csv("Results/Filter/"+ tipe +"-ner.csv")
    sf.to_excel("Results/Filter/"+ tipe +"-ner.xlsx")

In [11]:
run_filter("lda-partial", 'add_noun/', ner_ner)

In [12]:
run_filter("lda2vec-partial", 'add_noun/', ner_ner)

In [13]:
run_filter("plsa-partial", 'add_noun/', ner_ner)

In [14]:
run_filter("dp-partial", 'add_noun/', ner_ner)

In [15]:
import string
def rem_punct(word):
    return word.translate(str.maketrans('', '', string.punctuation))

In [19]:
import pandas as pd

def run_price(tipe, where):
    df = pd.read_csv("Results/" + where + tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'label','predict', 'term', 'polarity'])
    for index, reviews in enumerate(df['review']):
        tokenized = reviews.lower().replace('  ', ' ').strip().split(' ')
        predict = df['predict'][index]
        if type(predict) == float:
            predict = ''
        ner_word = ''
        flag = True
        for token in tokenized:
            if flag:
                for money in mon_ner:
                    if money in token:
                        if predict == '':
                            predict += 'PRICES'
                        else:
                            predict += '|PRICES'
                        ner_word = token
                        flag = False
                        break
        terms = []
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
        if ner_word != '':
            terms.append(ner_word)
        print('|'.join(terms))
        sentence = df['review'][index].lower()
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'target': df['target'][index],
                        'label': df['label'][index],
                        'predict': predict,
                        'term': '|'.join(terms),
                        'polarity': df['polarity'][index]
                       }, ignore_index=True)

    sf.to_csv("Results/Categorization/"+ tipe +"-money.csv")
    sf.to_excel("Results/Categorization/"+ tipe +"-money.xlsx")

In [48]:
run_price('plsa-partial-ner', 'Categorization/')

staff|place|like
food
food|good|always
always
good|wine
cart|attendant|lotus|leaf
food|great
chow|pork|table|family
price|service
service|excellent|always|decor
special
food
special
though
place|make|really|decor
sake|list|haze|request
good|great
service|great
food
service|friendly
ambiance
food
service
like|wait|make
food|great
service
restaurant|sushi|sushi
fresh|sushi
good
price|sushi|worth
food|service
menu|worth
food|fresh|$6
great
food|good|like|wait
like
service|nice
always
food|like
nice|ambience|would|best
idea|view|chart|house
food|price|service|great|great|great|reasonable|even
lava|cake|dessert
cosette|path|bistro
restaurant
food
food|ambience
pizza|even
price|pizza
good|pizza|make|even
meal|best
food|good|great|always|wine|menu
delicious|really|fresh
great|best
wife|shrimp
food|staff|wait|menu|special|even
good|atmosphere|special
staff|friendly|always|always
price|wine|reasonable
food|service|great|excellent|delicious|decor
point
food
food|service|great|place|always|wine
w

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [21]:
ate = ['lda', 'lda2vec', 'plsa', 'dp']
sim = ['glove', 'w2v', 'fasttext', 'elmo']

for a in ate:
    for s in sim:
        run_price(a + '-partial-ner-' + s, 'Categorization/')

place
food|portions
food|times
confit|duck
list|wine|values
cart|attendant|lotus|leaf
pork|chow|family
price
service|perfection
visit
stuff|food|fusion|thing
thing|chicken|puree
salt
effort
list|sake
roll|spicy|tempura|tuna|appetizer|rock|shrimp
service

service
ambiance
taste|mouth|service
self-respect
food
notch|service
sushi|restaurant|urchin|sushi
sushi
ration
advanatage|half|price|sushi|deal
food|impecable
menu|prix|fixe|penny
food|$6
dishes
food
somosas|chai|chole|dhosas|dhal
service|varys
kitchen
food
ambience|lunch|conversation
idea|view|chart|house
service|food|price|beginning
dessert|lava|cake
bistro|path
restaurant
desserts|soup|food
ambience|food|trip|world
pizza|cheese
pizza
ingredients|pizza
meals|life|minnow
seafood|wine|list|menu|surprises
combination|ingredients|dishes
portion|pastrami
shrimp|wife
signs|menus|food|waitstaff
leon|fare|bistro|village|specials|atmosphere
wife|staff
wine|prices
service|decor|food|portions
point
oily|food
food|wine|times|place|service
wait|

wait|trouble
glass|wine
service
people
pizza
pizza
toppings|$20,
lunch|reviews
sauce|flavor|food
food
service
pizza|service
wait
block
sushi|hands
food|place
tuna|gari
food
variety|menu
atmosphere
food
dumplings|section
sashimi|rolls
tuna
terrace|garden
food|service
tartare|steak
restaurant|kitchen|part
selection|wine|value|price|champagne|house
vibe|food
bread|tastes|pizza|pizza|toppings
neighborhood|food
setting|service|occasion
roll|scallop
moules|ravioli
maitre
dessert|$8.00...just
drinks
specials
lounge
restaurant
combinations|delight|food
staff
beers|wines
table|bathroom|dinner|orders|toilet
service|food
everythig
server
food
potatoes|tuna|wasabe
atmosphere
service
service|food
pizza|mozzarella|cheese|pizzaria
selection|appetizer|complaints
staff|business
food|restaurant
quality|dining|restaurant
cheeseburger|excpetiona|chicken|service
staff
food|options|rule
bagels
roll|heaven
sake|menu
roll|teriyaki
service
food|experience
selection|wine|list
decour|hall|restaurant
seating
food

decor|boths|side|side|hall
band|atmosphere|jazz
meal
food|diamond|balance|herbs|tomatoes|homemade
server
laugh|bathrooms|decor
husband|tortelini|beers|food|dish
bagel
workers|bagel|cream|cheese
frist|cafe|food|service
roll|mayo|mayo
guests|pizza|fish|chips
cafe|service|food
staff
chow|chow|oily
dumplings|pancakes
thing|service|restaurant
sushi|tuna|average|rolls|inside|flavor
penang|everything
price|service
restaurant|food|course
hats
salads
fish
vibe|lots
atmosphere
ambience
salads
salads
ingredients
sushi
roll|spicy|tuna
yellowtail
choices|food|$18
price|food|service
atmosphere
price|$3-6
takeout|tupperware|quality
caesar
pasta|dish
tiramisu|chocolate|cake
food|place
visit
food|service
food|service
dishes|wine|atmosphere
place|pizza|pizza
mine|goat|cheese|panchetta|raddichio|kind
friend|mushroom|pizza
sangria
beer
service|people
place
food
pizza
ambience
food
tuna|hanger
service|ambience
food|service
dish|mignon
restaurant
ambience
sushi

section|menu|nothing
meal
place|spot
service|

service|pizza|wine|wine|area|backyard|dining|place
place
menu|restaurant
wines|$70+
service|joint|price|category
service|grazie
factor|food
rosemary|flavoring|orange
fish
place|staff|wait
someone|dessert
expresso
service|service|fish
tuna|tartar|appetizer
family|pizzeria|owners|pizza|blows|families|receipies
place|service
food|atmosphere|service|service
room|dining|music
service|meal|menu
view|service|notch|wonderfull
service|staff
service|place|occassion
food|menu|price|price|allergies|prixe|fixe|tasting
place
staff
place
experience
service
drinks
service
delivery
resturant|prices
ambiance|boot
ambience|service
nothing|atmosphere|establishment
staff
prices|quality
trattoria
prices|gods|secret
room|secret
everytime|food|service|value
service
staff
service
staff

dessert|bill|chance|order
place|food
dishes
decor|service
waitstaffs
times|service
service
group|times
prices
food|portions
sandwiches|anything|chips|side|$7)
experience|desk|atmosphere
prices
seats
restaurant
cigar|staff
waits

In [12]:
'|'.join(['sd'])

'sd'